In [ ]:
# import the necessary modules

import networkx as nx
import blockmodeling as bm
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import scipy.cluster.hierarchy as sch

In [ ]:
# Specify the sociomatrix file you want to work with
#datafile="bm_examples/bakerCitation_el2.txt"
#datafile="bm_examples/simpleCP_el2.txt"
#datafile="bm_examples/trivial6actor_el2.txt"
datafile="bm_examples/LittleLeagueTI_el2.txt"

# Read edgelist from file, directional and binary
G = nx.read_adjlist(datafile,create_using=nx.DiGraph())

# Convert to matrix (numpy.matrix)
mat=nx.adjacency_matrix(G).todense()

# Extract the node labels into a list - useful later on
nodelabels=list(G.nodes())

# Display the sociomatrix
bm.displaySociomatrix(mat,nodelabels)

In [ ]:
# Display the network (you might get a warning message the first time - then do it again)
nx.draw(G,with_labels=True,node_color='#FFA0A0', node_size=500)
plt.show()

In [ ]:
# Calculate indirect structural equivalence (Hamming distances)
dist=bm.indirectSEhamming(mat) # Also works with bm.indirectSE(mat,method='hamming')

# or calculate correlation-based indirect structural equivalence instead
#corr=bm.indirectSEcorr(mat) # Also works with bm.indirectSE(mat,method='corr')
# As the clustering functionality here works for distances, we need to then convert correlations to distances
# I have added a function for this: corr2dist(mat) - so if you do the following:
#dist=bm.corr2dist(corr)
# ...you should have something that will work as we continue

In [ ]:
# Display the distance matrix if you want
#print(dist)

In [ ]:
# In order to do hierarchical clustering on these distance values, this must first be converted to a condensed distance matrix
# For this, we use scipy.spatial.distance.squareform:

dist_cond = sc.spatial.distance.squareform(dist)

In [ ]:
# When doing (agglomerative/bottom-up) hierarchical clustering, we can use one out of many different methods for clustering
# 'single-link' is typically NOT recommended; 'complete', 'average' and 'ward' are more common in this context

# Using the (non-weighted) average clustering approach, we create our clustering object
Z=sch.linkage(dist_cond, method='complete')

In [ ]:
# Then we can plot the dendrogram for this particular hierarchical clustering
plt.figure(figsize=(10, 7))
sch.dendrogram(Z,labels=nodelabels) # ...using the nodelabels we extracted in the beginning
plt.show()

In [ ]:
# The dendrogram should assist us in choosing a suitable cutoff in the dendrogram
# This cutoff will thus specify the number of positions our blockmodel will have,
# and which nodes/actors that will be part of different partitions


# We use the fcluster function to "cut" the dendrogram at a suitable level. First set that threshold/cutoff value:
threshold=10

partition = sch.fcluster(Z,threshold,'distance')

# This 'partition' object is a 1-dimensional array (numpy.ndarray), indicating (for each node) which position it belongs to

# Have a look at it:
print(partition)

In [ ]:
# Check that you got the number of partitions that you wanted
len(np.unique(partition))

In [ ]:
# An alternative (and better) way to store this partition is in the form of a dictionary, of lists
# In the blockmodeling library, I have created a function that generates such a dictionary, from your partition list

blockdict=bm.createBlockdict(partition)

# Display it and you will see
bm.displayBlockdict(blockdict,nodelabels)

In [ ]:
# Finally, given your original sociomatrix (max),
# the partition you have found (blockdict), and
# the nodelabels, you can display the final blockmodel

bm.displayBlockmodel(mat,blockdict,nodelabels)

In [ ]:
# To assist in interpreting this, I have created a function for
# calculating block densities. It returns a numpy ndarray, containing the
# block densities for the partition specified in blockdict
# This blockimage is sorted according to the indices of blockdict,
# i.e. following the same order as the blockmodel above

densityBI=bm.calcDensityBlockimage(mat,blockdict)

print(np.around(densityBI,decimals=2))

In [ ]:
# Values below 0.5 are thus closer to a null block
# and values above 0.5 are closer to a complete block

# But ideally, these values should be either close to zero or unity